# TMDB movies - random forest regressor

In [0]:
import numpy as np 
import pandas as pd 
import category_encoders as ce 

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline


# Load Data

In [0]:
dataPath = "/FileStore/tables/tmdb_movies_data.csv"
movies = sqlContext.read.format("csv").option("header","true").option("inferSchema", "true").load(dataPath)
movies = movies.withColumnRenamed("cast", "actors")
pmovies = movies.toPandas()

pmovies.head()

Out[60]:

,id,imdb_id,popularity,budget,revenue,original_title,actors,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,monster|dna|tyrannosaurus rex|velociraptor|island,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1392445893
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,future|chase|post-apocalyptic|dystopia|australia,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,348161292.5
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,based on novel|revolution|dystopia|sequel|dyst...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,271619025.4
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,android|spaceship|jedi|space opera|3d,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919,1902723130
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,car race|speed|revenge|suspense|car,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1385748801


In [0]:
pmovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10879 entries, 0 to 10878
Data columns (total 21 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 id 10879 non-null object
 1 imdb_id 10869 non-null object
 2 popularity 10879 non-null object
 3 budget 10877 non-null object
 4 revenue 10876 non-null object
 5 original_title 10875 non-null object
 6 actors 10799 non-null object
 7 homepage 2946 non-null object
 8 director 10832 non-null object
 9 tagline 8052 non-null object
 10 keywords 9381 non-null object
 11 overview 10868 non-null object
 12 runtime 10860 non-null object
 13 genres 10834 non-null object
 14 production_companies 9923 non-null object
 15 release_date 10837 non-null object
 16 vote_count 10831 non-null object
 17 vote_average 10844 non-null object
 18 release_year 10845 non-null object
 19 budget_adj 10855 non-null object
 20 revenue_adj 10848 non-null object
dtypes: object(21)
memory usage: 1.7+ MB

In [0]:
movies.createOrReplaceTempView("movies")

In [0]:
%sql 
drop table if exists movies_cleaned; 

create table movies_cleaned as 
select 
int(id) id, 
imdb_id, 
double(popularity) popularity, 
double(budget) budget, 
double(revenue) revenue, 
int(runtime) runtime, 
int(vote_count) votes, 
int(vote_average) votes_avg, 
actors,
genres,
tagline,      
double(budget_adj) budget_ad, 
double(revenue_adj) revenue_ad
from movies 


In [0]:
sqlContext.cacheTable("movies_cleaned")
pmovies = spark.sql("SELECT * FROM movies_cleaned").toPandas()

#drop rows will null values 
pmovies = pmovies.dropna()


In [0]:

#feature engineering 
#pmovies['actors_count'] = pmovies['actors'].str.findall('|').str.len() + 1
pmovies['actors_count'] = pmovies['actors'][0].count('|') + 1
pmovies['genre_count'] = pmovies['genres'][0].count('|') + 1 
pmovies['has_tagline'] = pmovies['tagline'].apply(lambda x: 1 if x is not None else 0)




pmovies = pmovies[(pmovies['budget_ad'] > 0) | (pmovies['revenue_ad'] > 0) ]

pmovies.info()

#pmovies[pmovies.has_tagline == 0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5010 entries, 0 to 10878
Data columns (total 16 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 id 5010 non-null float64
 1 imdb_id 5010 non-null object 
 2 popularity 5010 non-null float64
 3 budget 5010 non-null float64
 4 revenue 5010 non-null float64
 5 runtime 5010 non-null float64
 6 votes 5010 non-null float64
 7 votes_avg 5010 non-null float64
 8 actors 5010 non-null object 
 9 genres 5010 non-null object 
 10 tagline 5010 non-null object 
 11 budget_ad 5010 non-null float64
 12 revenue_ad 5010 non-null float64
 13 actors_count 5010 non-null int64 
 14 genre_count 5010 non-null int64 
 15 has_tagline 5010 non-null int64 
dtypes: float64(9), int64(3), object(4)
memory usage: 665.4+ KB

In [0]:
prepped = spark.createDataFrame(pmovies) #.na.fill(0)
display(prepped)

id,imdb_id,popularity,budget,revenue,runtime,votes,votes_avg,actors,genres,tagline,budget_ad,revenue_ad,actors_count,genre_count,has_tagline
135397.0,tt0369610,32.985763,1.5E8,1.51352881E9,124.0,5562.0,6.0,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vincent D'Onofrio|Nick Robinson,Action|Adventure|Science Fiction|Thriller,The park is open.,1.379999393E8,1.392445893E9,5,4,1
76341.0,tt1392190,28.419936,1.5E8,3.78436354E8,120.0,6185.0,7.0,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nicholas Hoult|Josh Helman,Action|Adventure|Science Fiction|Thriller,What a Lovely Day.,1.379999393E8,3.481612925E8,5,4,1
262500.0,tt2908446,13.112507,1.1E8,2.95238201E8,119.0,2480.0,6.0,Shailene Woodley|Theo James|Kate Winslet|Ansel Elgort|Miles Teller,Adventure|Science Fiction|Thriller,One Choice Can Destroy You,1.011999555E8,2.716190254E8,5,4,1
140607.0,tt2488496,11.173104,2.0E8,2.068178225E9,136.0,5292.0,7.0,Harrison Ford|Mark Hamill|Carrie Fisher|Adam Driver|Daisy Ridley,Action|Adventure|Science Fiction|Fantasy,Every generation has a story.,1.83999919E8,1.90272313E9,5,4,1
168259.0,tt2820852,9.335014,1.9E8,1.50624936E9,137.0,2947.0,7.0,Vin Diesel|Paul Walker|Jason Statham|Michelle Rodriguez|Dwayne Johnson,Action|Crime|Thriller,Vengeance Hits Home,1.747999231E8,1.385748801E9,5,4,1
281957.0,tt1663202,9.1107,1.35E8,5.32950503E8,156.0,3929.0,7.0,Leonardo DiCaprio|Tom Hardy|Will Poulter|Domhnall Gleeson|Paul Anderson,Western|Drama|Adventure|Thriller,"(n. One who has returned, as if from the dead.)",1.241999454E8,4.90314247E8,5,4,1
87101.0,tt1340138,8.654359,1.55E8,4.40603537E8,125.0,2598.0,5.0,Arnold Schwarzenegger|Jason Clarke|Emilia Clarke|Jai Courtney|J.K. Simmons,Science Fiction|Action|Thriller|Adventure,Reset the future,1.425999373E8,4.053550757E8,5,4,1
286217.0,tt3659388,7.6674,1.08E8,5.95380321E8,141.0,4572.0,7.0,Matt Damon|Jessica Chastain|Kristen Wiig|Jeff Daniels|Michael PeÃ±a,Drama|Adventure|Science Fiction,Bring Him Home,9.935995628E7,5.477496543E8,5,4,1
211672.0,tt2293640,7.404165,7.4E7,1.156730962E9,91.0,2893.0,6.0,Sandra Bullock|Jon Hamm|Michael Keaton|Allison Janney|Steve Coogan,Family|Animation|Adventure|Comedy,"Before Gru, they had a history of bad bosses",6.807997004E7,1.064192017E9,5,4,1
150540.0,tt2096673,6.326804,1.75E8,8.53708609E8,94.0,3935.0,8.0,Amy Poehler|Phyllis Smith|Richard Kind|Bill Hader|Lewis Black,Comedy|Animation|Family,Meet the little voices inside your head.,1.609999292E8,7.854115747E8,5,4,1


In [0]:
import random
random.seed(10)

# Using Base line Attributes

In [0]:
nonFeatureCols = ["id", "imdb_id", "budget", "revenue", "actors", "genres", "tagline", "actors_count", "genre_count", "has_tagline"]
featureCols = [item for item in prepped.columns if item not in nonFeatureCols]
featureCols

Out[67]: ['popularity', 'runtime', 'votes', 'votes_avg', 'budget_ad', 'revenue_ad']

In [0]:
# VectorAssembler Assembles all of these columns into one single vector. To do this, set the input columns and output column. Then that assembler will be used to transform the prepped data to the final dataset.
from pyspark.ml.feature import VectorAssembler

assembler = (VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features"))

finalPrep = assembler.transform(prepped)
training, test = finalPrep.randomSplit([0.7, 0.3])

#  Going to cache the data to make sure things stay snappy!
training.cache()
test.cache()

print(training.count()) 
print(test.count())

3502
1508

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

rfModel = (RandomForestRegressor()
  .setLabelCol("revenue_ad")
  .setFeaturesCol("features"))

paramGrid = (ParamGridBuilder()
  .addGrid(rfModel.maxDepth, [3, 5, 8])
  .addGrid(rfModel.numTrees, [5, 10, 15])
  .build())
  

stages = [rfModel]

pipeline = Pipeline().setStages(stages)

cv = (CrossValidator() # you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline) # the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid)
  .setEvaluator(RegressionEvaluator().setLabelCol("revenue_ad")))

pipelineFitted = cv.fit(training)

print("The Best Parameters:\n--------------------")
print(pipelineFitted.bestModel.stages[0])
pipelineFitted.bestModel.stages[0].extractParamMap()

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
The Best Parameters:
--------------------
RandomForestRegressionModel: uid=RandomForestRegressor_6dd8d2b1ceb1, numTrees=15, numFeatures=6
Out[69]: {Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"): 'auto',
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='labelCol', doc='label column name.'): 'revenue_ad',
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='leafCol', doc='Leaf indices column name. Predicted leaf index of each instance in each tree by preorder.'): '',
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'): 256,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='minWeightFractionPerNode', doc='Minimum fraction of the weighted sample count that each child must have after split. If a split causes the fraction of the total weight in the left or right child to be less than minWeightFractionPerNode, the split will be discarded as invalid. Should be in interval [0.0, 0.5).'): 0.0,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='numTrees', doc='Number of trees to train (>= 1).'): 15,
 Param(parent='RandomForestRegressor_6dd8d2b1ceb1', name='predictionCol', doc='prediction

In [0]:
holdout = (pipelineFitted.bestModel
  .transform(test)
  .selectExpr("prediction as raw_prediction", "double(revenue_ad)", "(prediction/double(revenue_ad))*100 as perc_diff"))
  
#display(holdout)

from pyspark.mllib.evaluation import RegressionMetrics
rm3 = RegressionMetrics(holdout.select("raw_prediction", "revenue_ad").rdd.map(lambda x:  (x[0], x[1])))

print("MSE: ", rm3.meanSquaredError)
print("MAE: ", rm3.meanAbsoluteError)
print("RMSE Squared: ", rm3.rootMeanSquaredError)
print("R Squared: ", rm3.r2)
print("Explained Variance: ", rm3.explainedVariance, "\n")

/databricks/spark/python/pyspark/sql/context.py:120: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)
MSE: 4166823788521779.0
MAE: 20525311.00591374
RMSE Squared: 64550939.48597324
R Squared: 0.8838628866480306
Explained Variance: 3.1164589914572228e+16

# actor count

In [0]:
nonFeatureCols = ["id", "imdb_id", "budget", "revenue", "actors", "genres", "tagline", "genre_count", "has_tagline"]
featureCols = [item for item in prepped.columns if item not in nonFeatureCols]
featureCols

Out[71]: ['popularity',
 'runtime',
 'votes',
 'votes_avg',
 'budget_ad',
 'revenue_ad',
 'actors_count']

In [0]:
# VectorAssembler Assembles all of these columns into one single vector. To do this, set the input columns and output column. Then that assembler will be used to transform the prepped data to the final dataset.
from pyspark.ml.feature import VectorAssembler

assembler = (VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features"))

finalPrep = assembler.transform(prepped)
training, test = finalPrep.randomSplit([0.7, 0.3])

#  Going to cache the data to make sure things stay snappy!
training.cache()
test.cache()

print(training.count()) 
print(test.count())

3465
1545

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

rfModel = (RandomForestRegressor()
  .setLabelCol("revenue_ad")
  .setFeaturesCol("features"))

paramGrid = (ParamGridBuilder()
  .addGrid(rfModel.maxDepth, [3, 5, 8])
  .addGrid(rfModel.numTrees, [5, 10, 15])
  .build())
  

stages = [rfModel]

pipeline = Pipeline().setStages(stages)

cv = (CrossValidator() # you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline) # the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid)
  .setEvaluator(RegressionEvaluator().setLabelCol("revenue_ad")))

pipelineFitted = cv.fit(training)

print("The Best Parameters:\n--------------------")
print(pipelineFitted.bestModel.stages[0])
pipelineFitted.bestModel.stages[0].extractParamMap()

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
The Best Parameters:
--------------------
RandomForestRegressionModel: uid=RandomForestRegressor_4c3a3e486f7d, numTrees=15, numFeatures=7
Out[73]: {Param(parent='RandomForestRegressor_4c3a3e486f7d', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"): 'auto',
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='labelCol', doc='label column name.'): 'revenue_ad',
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='leafCol', doc='Leaf indices column name. Predicted leaf index of each instance in each tree by preorder.'): '',
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'): 256,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='minWeightFractionPerNode', doc='Minimum fraction of the weighted sample count that each child must have after split. If a split causes the fraction of the total weight in the left or right child to be less than minWeightFractionPerNode, the split will be discarded as invalid. Should be in interval [0.0, 0.5).'): 0.0,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='numTrees', doc='Number of trees to train (>= 1).'): 15,
 Param(parent='RandomForestRegressor_4c3a3e486f7d', name='predictionCol', doc='prediction

In [0]:
holdout = (pipelineFitted.bestModel
  .transform(test)
  .selectExpr("prediction as raw_prediction", "double(revenue_ad)", "(prediction/double(revenue_ad))*100 as perc_diff"))
  
#display(holdout)

from pyspark.mllib.evaluation import RegressionMetrics
rm3 = RegressionMetrics(holdout.select("raw_prediction", "revenue_ad").rdd.map(lambda x:  (x[0], x[1])))

print("MSE: ", rm3.meanSquaredError)
print("MAE: ", rm3.meanAbsoluteError)
print("RMSE Squared: ", rm3.rootMeanSquaredError)
print("R Squared: ", rm3.r2)
print("Explained Variance: ", rm3.explainedVariance, "\n")

MSE: 2940019833210588.5
MAE: 14946198.48921256
RMSE Squared: 54221949.73634376
R Squared: 0.9071315173973455
Explained Variance: 2.985972496267101e+16

# Genre Count

In [0]:
nonFeatureCols = ["id", "imdb_id", "budget", "revenue", "actors", "genres", "tagline", "has_tagline"]
featureCols = [item for item in prepped.columns if item not in nonFeatureCols]
featureCols

Out[75]: ['popularity',
 'runtime',
 'votes',
 'votes_avg',
 'budget_ad',
 'revenue_ad',
 'actors_count',
 'genre_count']

In [0]:
# VectorAssembler Assembles all of these columns into one single vector. To do this, set the input columns and output column. Then that assembler will be used to transform the prepped data to the final dataset.
from pyspark.ml.feature import VectorAssembler

assembler = (VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features"))

finalPrep = assembler.transform(prepped)
training, test = finalPrep.randomSplit([0.7, 0.3])

#  Going to cache the data to make sure things stay snappy!
training.cache()
test.cache()

print(training.count()) 
print(test.count())

3526
1484

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

rfModel = (RandomForestRegressor()
  .setLabelCol("revenue_ad")
  .setFeaturesCol("features"))

paramGrid = (ParamGridBuilder()
  .addGrid(rfModel.maxDepth, [3, 5, 8])
  .addGrid(rfModel.numTrees, [5, 10, 15])
  .build())
  

stages = [rfModel]

pipeline = Pipeline().setStages(stages)

cv = (CrossValidator() # you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline) # the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid)
  .setEvaluator(RegressionEvaluator().setLabelCol("revenue_ad")))

pipelineFitted = cv.fit(training)

print("The Best Parameters:\n--------------------")
print(pipelineFitted.bestModel.stages[0])
pipelineFitted.bestModel.stages[0].extractParamMap()

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
The Best Parameters:
--------------------
RandomForestRegressionModel: uid=RandomForestRegressor_9bb68d06dbcc, numTrees=15, numFeatures=8
Out[77]: {Param(parent='RandomForestRegressor_9bb68d06dbcc', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"): 'auto',
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='labelCol', doc='label column name.'): 'revenue_ad',
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='leafCol', doc='Leaf indices column name. Predicted leaf index of each instance in each tree by preorder.'): '',
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'): 256,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='minWeightFractionPerNode', doc='Minimum fraction of the weighted sample count that each child must have after split. If a split causes the fraction of the total weight in the left or right child to be less than minWeightFractionPerNode, the split will be discarded as invalid. Should be in interval [0.0, 0.5).'): 0.0,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='numTrees', doc='Number of trees to train (>= 1).'): 15,
 Param(parent='RandomForestRegressor_9bb68d06dbcc', name='predictionCol', doc='prediction

In [0]:
holdout = (pipelineFitted.bestModel
  .transform(test)
  .selectExpr("prediction as raw_prediction", "double(revenue_ad)", "(prediction/double(revenue_ad))*100 as perc_diff"))
  
#display(holdout)

from pyspark.mllib.evaluation import RegressionMetrics
rm3 = RegressionMetrics(holdout.select("raw_prediction", "revenue_ad").rdd.map(lambda x:  (x[0], x[1])))

print("MSE: ", rm3.meanSquaredError)
print("MAE: ", rm3.meanAbsoluteError)
print("RMSE Squared: ", rm3.rootMeanSquaredError)
print("R Squared: ", rm3.r2)
print("Explained Variance: ", rm3.explainedVariance, "\n")

MSE: 2852685261048046.5
MAE: 16816978.6285293
RMSE Squared: 53410535.11291613
R Squared: 0.9114717682209467
Explained Variance: 3.0308207087100444e+16

#  has a tag line

In [0]:
nonFeatureCols = ["id", "imdb_id", "budget", "revenue", "actors", "genres", "tagline"]
featureCols = [item for item in prepped.columns if item not in nonFeatureCols]
featureCols

Out[83]: ['popularity',
 'runtime',
 'votes',
 'votes_avg',
 'budget_ad',
 'revenue_ad',
 'actors_count',
 'genre_count',
 'has_tagline']

In [0]:
# VectorAssembler Assembles all of these columns into one single vector. To do this, set the input columns and output column. Then that assembler will be used to transform the prepped data to the final dataset.
from pyspark.ml.feature import VectorAssembler

assembler = (VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features"))

finalPrep = assembler.transform(prepped)
training, test = finalPrep.randomSplit([0.7, 0.3])

#  Going to cache the data to make sure things stay snappy!
training.cache()
test.cache()

print(training.count()) 
print(test.count())

3492
1518

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

rfModel = (RandomForestRegressor()
  .setLabelCol("revenue_ad")
  .setFeaturesCol("features"))

paramGrid = (ParamGridBuilder()
  .addGrid(rfModel.maxDepth, [3, 5, 8])
  .addGrid(rfModel.numTrees, [5, 10, 15])
  .build())
  

stages = [rfModel]

pipeline = Pipeline().setStages(stages)

cv = (CrossValidator() # you can feel free to change the number of folds used in cross validation as well
  .setEstimator(pipeline) # the estimator can also just be an individual model rather than a pipeline
  .setEstimatorParamMaps(paramGrid)
  .setEvaluator(RegressionEvaluator().setLabelCol("revenue_ad")))

pipelineFitted = cv.fit(training)

print("The Best Parameters:\n--------------------")
print(pipelineFitted.bestModel.stages[0])
pipelineFitted.bestModel.stages[0].extractParamMap()

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
The Best Parameters:
--------------------
RandomForestRegressionModel: uid=RandomForestRegressor_927f2b8a8d0e, numTrees=15, numFeatures=9
Out[85]: {Param(parent='RandomForestRegressor_927f2b8a8d0e', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"): 'auto',
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='labelCol', doc='label column name.'): 'revenue_ad',
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='leafCol', doc='Leaf indices column name. Predicted leaf index of each instance in each tree by preorder.'): '',
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='maxBins', doc='Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 8,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'): 256,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='minWeightFractionPerNode', doc='Minimum fraction of the weighted sample count that each child must have after split. If a split causes the fraction of the total weight in the left or right child to be less than minWeightFractionPerNode, the split will be discarded as invalid. Should be in interval [0.0, 0.5).'): 0.0,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='numTrees', doc='Number of trees to train (>= 1).'): 15,
 Param(parent='RandomForestRegressor_927f2b8a8d0e', name='predictionCol', doc='prediction

In [0]:
holdout = (pipelineFitted.bestModel
  .transform(test)
  .selectExpr("prediction as raw_prediction", "double(revenue_ad)", "(prediction/double(revenue_ad))*100 as perc_diff"))
  
#display(holdout)

from pyspark.mllib.evaluation import RegressionMetrics
rm3 = RegressionMetrics(holdout.select("raw_prediction", "revenue_ad").rdd.map(lambda x:  (x[0], x[1])))

print("MSE: ", rm3.meanSquaredError)
print("MAE: ", rm3.meanAbsoluteError)
print("RMSE Squared: ", rm3.rootMeanSquaredError)
print("R Squared: ", rm3.r2)
print("Explained Variance: ", rm3.explainedVariance, "\n")

MSE: 4500276574331195.0
MAE: 22606498.435570247
RMSE Squared: 67084100.75667106
R Squared: 0.8832637512222834
Explained Variance: 2.5457385176517852e+16